<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание" data-toc-modified-id="Задание-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Rules" data-toc-modified-id="Rules-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Rules</a></span><ul class="toc-item"><li><span><a href="#Greeting" data-toc-modified-id="Greeting-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Greeting</a></span></li><li><span><a href="#Meeting" data-toc-modified-id="Meeting-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Meeting</a></span><ul class="toc-item"><li><span><a href="#Manager's-name" data-toc-modified-id="Manager's-name-1.3.2.1"><span class="toc-item-num">1.3.2.1&nbsp;&nbsp;</span>Manager's name</a></span></li></ul></li><li><span><a href="#Company" data-toc-modified-id="Company-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Company</a></span></li><li><span><a href="#Bye" data-toc-modified-id="Bye-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Bye</a></span></li></ul></li><li><span><a href="#ManagerChecker" data-toc-modified-id="ManagerChecker-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>ManagerChecker</a></span></li></ul></li></ul></div>

# Задание


Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. 
Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 
Главные задачи, которые должен выполнять скрипт:
1. Извлекать реплики с приветствием – где менеджер поздоровался. 
2. Извлекать реплики, где менеджер представил себя. 
3. Извлекать имя менеджера. 
4. Извлекать название компании. 
5. Извлекать реплики, где менеджер попрощался.
6. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


Рекомендации:
* Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
* Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
* Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
* Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей.



## Импорт библиотек

In [1]:
import pandas as pd
from natasha import NamesExtractor, MorphVocab
from ipymarkup import show_span_box_markup as show_markup
from yargy import rule, Parser, or_
from yargy.predicates import eq, gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact

## Знакомство с данными

In [2]:
df = pd.read_csv('test_data.csv')

df.head(10)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
6,0,6,client,Да мы работаем с компанией которая нам подлива...
7,0,7,client,Как как бы уже до этого момента работаем все у...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
9,0,9,client,Как бы нет


In [3]:
# Номера уникальных диалогов

df.dlg_id.unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [4]:
# Уникальные роли

df.role.unique()

array(['client', 'manager'], dtype=object)

## Rules

### Greeting
Реплики с приветствием – где менеджер поздоровался. 

In [5]:
TIMES = morph_pipeline([
    'утро',
    'день',
    'вечер',
    'ночь'
])

ADJF_G = morph_pipeline([
    'добрый'
])


GREETING = rule(or_(
    morph_pipeline(['здравствуйте']),
    rule(ADJF_G,
        TIMES.optional())))

### Meeting
Реплики, где менеджер представил себя.

In [6]:
NAMING = morph_pipeline([
    'зовут',
    'меня зовут', 
    'моё имя'])

OTHER_M = morph_pipeline(['да это'])


MEETING = rule(or_(NAMING, OTHER_M))


#### Manager's name
Извлекать имя менеджера.

In [7]:
morph_vocab = MorphVocab()

### Company
Извлекать название компании.

In [8]:
Company = fact(
    'Company',
    ['company_name'])


COMPANY = morph_pipeline([
    'диджитал бизнес',
    'китобизнес']
).interpretation(Company.company_name)

### Bye
Извлекать реплики, где менеджер попрощался.

In [9]:
ADJF_B = morph_pipeline([
    'добрый',
    'хороший'])


BYE = rule(or_(morph_pipeline(['до свидания']),
               rule(eq('всего'), ADJF_B),
               rule(ADJF_B,TIMES))) 

## ManagerChecker

In [10]:
class ManagerChecker:
    
    def __init__(self):
        self.greeting = Parser(GREETING)
        self.meeting = Parser(MEETING)
        self.extractor = NamesExtractor(morph_vocab)
        self.company = Parser(COMPANY)
        self.bye = Parser(BYE)
        self.name = {} # Словарь для хранения имен менеджера
        self.has_greeting = {} # Словарь для хранения факта было ли приветствие в диалоге
        self.has_bye = {}  # Словарь для хранения факта было ли прощания в диалоге
        self.checker = {} # Словарь для хранения факта было ли ПРОЩАНИЕ И ПРИВЕТСТВИЕ  в диалоге
        self.name_company = {} # Словарь для хранения названия компании
        
    
    
    def __call__(self, df):
        for  dlg_id  in df.dlg_id.unique():
            self.parser(df, dlg_id)
        
        
    
    @staticmethod    
    def preprocessing(df, dlg_id, start_dlg):
        """preprocessing - функция, которая подготавливает данные к парсингу:
            1. Исключаем реплики клиента;
            2. Выделяем реплики, которые относятся к одному диалогу;
            3. Берём колонку c репликами - text
            4. В зависимости от контекста берём ту часть диалога, где более вероятно встретить ту или иную реплику.
                Например: Приветствие будут искать в начале диалога, прощание - в конце.
                Это нужно для того, чтобы скрипт быстрее отрабатывал.
            5. Приведение к нижнему регистру"""
        if start_dlg:
            df_ = df[(df['role'] == 'manager') &
                         (df['dlg_id'] == dlg_id)]['text'][:10]
            return df_.apply(lambda _: _.lower())
            
        else:
            df_ = df[(df['role'] == 'manager') &
                         (df['dlg_id'] == dlg_id)]['text'][-10:]
            return df_.apply(lambda _: _.lower())
    
                  
                
    def get_greeting(self, df, dlg_id):
        """get_greeting - извлекать реплики с приветствием – где менеджер поздоровался из диалога.
           А также записывает has_greeting факт было ли приветствие в диалоге"""
        self.has_greeting[dlg_id] = False
        df_ = self.preprocessing(df,  dlg_id = dlg_id, start_dlg = True)
        for line in df_:
            matches =list(self.greeting.findall(line))
            spans = [_.span for _ in matches]
            if len(spans):
                self.has_greeting[dlg_id] = True
                show_markup(line, spans)
                
            
            
    def get_meeting(self, df, dlg_id):
        """get_meeting -  функция, которая извлекает реплики, в которых менеджер представил себя.
        Также из контекста дастаём имя менеджера"""
        self.name[dlg_id] = None
        df_ = self.preprocessing(df,  dlg_id=dlg_id, start_dlg = True)
        for line in df_:
            matches =list(self.meeting.findall(line))    
            spans = [_.span for _ in matches]
            if len(spans):
                match_name = self.extractor(line)
                for match in match_name:
                    start, stop = match.start, match.stop
                    if match.fact.first:
                        show_markup(line, spans)
                        self.name[dlg_id] = match.fact.first
       
        
            
    def get_company(self, df, dlg_id):
        """get_company - извлекает название компании"""
        self.name_company[dlg_id] = None
        df_ = self.preprocessing(df,  dlg_id =  dlg_id, start_dlg = True)
        for line in df_:
            matches =list(self.company.findall(line))    
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                self.name_company[dlg_id] = facts[0]
                display(self.name_company[dlg_id])
       
            
    
    def get_bye(self, df, dlg_id):
        """get_bye - извлекать реплики с прощанием.
           А также записывает в has_bye факт было ли прощание в диалоге"""
        self.has_bye[dlg_id] = False
        df_ = self.preprocessing(df,  dlg_id =  dlg_id, start_dlg = False)
        for line in df_:
            matches =list(self.bye.findall(line))    
            spans = [_.span for _ in matches]
            if len(spans):
                self.has_bye[dlg_id] = True
                show_markup(line, spans)
                
                
                
    def get_check(self, df,  dlg_id):
        """Проверка требование к менеджеру: 
        «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»
        Если два условия выполнены, вернётся True
        Если хотя бы одно условие не выполнено, вернётся False
        """
        self.checker[dlg_id] = self.has_greeting[dlg_id] and self.has_bye[dlg_id]
        return self.checker[dlg_id] 
    
    
                
    def parser(self, df, dlg_id):
        print(f'ДИАЛОГ НОМЕР {dlg_id}')
        print('Pеплики с приветствием – где менеджер поздоровался:')
        self.get_greeting(df,  dlg_id)
        print('Реплики, где менеджер представил себя :')
        self.get_meeting(df, dlg_id)
        print('Имя менеджера : ')
        display(self.name[dlg_id])
        print('Название компании :')
        self.get_company(df, dlg_id)
        print('Pеплики, где менеджер попрощался :')
        self.get_bye(df, dlg_id) 
        print('Проверка требование к менеджеру: «необходимо поздороваться и попрощаться с клиентом» :')
        print(self.get_check(df, dlg_id))
        print('=====================================================')
    

In [11]:
checker = ManagerChecker()
checker(df)

ДИАЛОГ НОМЕР 0
Pеплики с приветствием – где менеджер поздоровался:


Реплики, где менеджер представил себя :


Имя менеджера : 


'ангелина'

Название компании :


...

Проверка требование к менеджеру: «необходимо поздороваться и попрощаться с клиентом» :
False
ДИАЛОГ НОМЕР 5
Pеплики с приветствием – где менеджер поздоровался:
Реплики, где менеджер представил себя :


Имя менеджера : 


'анастасия'

Название компании :
Pеплики, где менеджер попрощался :


Проверка требование к менеджеру: «необходимо поздороваться и попрощаться с клиентом» :
False


In [12]:
# имена менеждеров

checker.name

{0: 'ангелина',
 1: 'ангелина',
 2: 'ангелина',
 3: 'максим',
 4: None,
 5: 'анастасия'}

In [13]:
# названия компаний

checker.name_company

{0: 'диджитал бизнес',
 1: 'диджитал бизнес',
 2: 'диджитал бизнес',
 3: 'китобизнес',
 4: None,
 5: None}

In [14]:
# Проверка требования к менеджеру

checker.checker

{0: True, 1: True, 2: False, 3: True, 4: False, 5: False}